<a href="https://colab.research.google.com/github/newfull5/AI-Project/blob/master/nlp_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb transformers datasets torch tqdm

In [2]:
!nvidia-smi

Fri Jan 20 07:37:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    57W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import argparse

def _get_parser():  
    parser = argparse.ArgumentParser()
    parser.add_argument()
    return parser
   

args = argparse.Namespace(  
  model_name="klue/bert-base", 
  tokenizer_name="klue/bert-base",
  dataset_name="nsmc",
  val_check_step=2000,
  batch_size=32,
  max_epochs=10,
  patient=3, 
  lr=3e-5
)

In [4]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from datasets import load_dataset

class NsmcDataset(Dataset):
  def __init__(self, args, stage):
    super().__init__()
    self.stage = stage
    self.args = args
    self.documents, self.labels = self._get_data(args, stage)

  def _get_data(self, args, stage):
    dataset = load_dataset(args.dataset_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)

    if stage == 'train':
      documents = dataset['train']['document']
      labels = dataset['train']['label']
    elif stage == 'valid':
      documents = dataset['test']['document'][:25000]
      labels = dataset['test']['label'][:25000]
    elif stage == 'test':
      documents = dataset['test']['document'][25000:]
      labels = dataset['test']['label'][25000:]
    else:
      raise Exception("you can set stage only 'train', 'test' or 'valid'")

    return [self._preproc(tokenizer, text) for text in documents], labels

  def _preproc(self, tokenizer, text):
    tokenized = tokenizer(
      text=text,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
    )
    return tokenized
      
  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.documents[idx], self.labels[idx]

In [5]:
from transformers import AutoModelForSequenceClassification
import torch
from torch import nn

class Model(nn.Module):
  def __init__(self, args):
    super(Model, self).__init__()
    self.model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=2)
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.criterion = nn.CrossEntropyLoss()
    self.model.to(self.device)

  def forward(self, batch):
    inputs, labels = batch
    inputs['input_ids'] = self._move_to_cuda(inputs['input_ids'].squeeze())
    inputs['attention_mask'] = self._move_to_cuda(inputs['attention_mask'])
    labels = self._move_to_cuda(labels)

    outputs = self.model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
    )
    loss = self.criterion(outputs.logits, labels)

    return outputs.logits, loss

  def _move_to_cuda(self, inputs):
    if torch.is_tensor(inputs):
      return inputs.to(self.device)
    elif isinstance(inputs, list):
      return [self._move_to_cuda(x) for x in inputs]
    elif isinstance(inputs, dict):
      return {key: self._move_to_cuda(value) for key, value in inputs.items()}
    else:
      return inputs

  def save(self, save_dir):
    self.model.save_pretrained(save_dir)

  def load(self, save_dir):
    self.model.load_state_dict(
        torch.load(f"{save_dir}/pytorch_model.bin", map_location=torch.device(self.device))
    )

In [6]:
from tqdm import tqdm
import wandb

class Trainer:
  def __init__(self, args, model, train_loader, valid_loader):
    self.model = model
    self.args = args
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    self.max_epochs = args.max_epochs
    self.val_check_step = args.val_check_step
    self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=args.lr)
    self.epoch = 0
    self.n_th = 0
    self.global_steps = 0

  def train_step(self):
    self.model.train()
    total_train_loss = 0
    train_steps = 0
    for step, batch in tqdm(enumerate(self.train_loader), desc=f'train epoch: {self.epoch}'):
      logit, loss = self.model(batch)
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      total_train_loss = float(loss)
      train_steps += 1
      self.global_steps += 1

      if step % self.val_check_step == 0 and step != 0:
        wandb.log({
            'train_loss': (total_train_loss/train_steps)
            })
        self.valid_step()

    else:
      self.epoch += 1

  def valid_step(self):
    self.model.eval()
    total_val_loss = 0
    val_steps = 0
    pred = []
    labels = []
    with torch.no_grad():
      for step, batch in enumerate(self.valid_loader):
        logits, loss = self.model(batch)
        total_val_loss += float(loss)
        val_steps += 1

        _, label = batch
        labels += label.tolist()
        pred += logits.argmax(dim=1).tolist()

    tp,tn,fp,fn = self._confusion_matrix(pred, labels)
    precision = self._calc_precision(tp,tn,fp,fn)
    recall = self._calc_recall(tp,tn,fp,fn)
    f1_score = 2 * precision * recall / (precision + recall)

    wandb.log({
        'val_loss': (total_val_loss/val_steps),
        'val_precision': precision,
        'val_recall': recall,
        'val_f1score': f1_score,
        'steps': self.global_steps
        })
    self.model.save(f"./nsmc_ckpt_steps_/{self.n_th}")
    self.n_th += 1
    
  def _confusion_matrix(self, pred, labels):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for i in range(len(labels)):
      if pred[i] == 1:
        if pred[i] == labels[i]:
          tp += 1
        if pred[i] != labels[i]:
          fp += 1

      elif pred[i] == 0:
        if pred[i] == labels[i]:
          tn += 1
        if pred[i] != labels[i]:
          fn += 1

    return tp, tn, fp, fn

  def _calc_precision(self, tp, tn, fp, fn):
    if tp + fn == 0:
      return 1
    return tp / (tp + fp)

  def _calc_recall(self, tp, tn, fp, fn):
    if tp + fn == 1:
      return 1
    return tp / (tp + fn)

  def fit(self):
    for epoch in range(self.args.max_epochs):
      self.train_step()

In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = NsmcDataset(args, 'train'),
    batch_size = args.batch_size,
    shuffle = True
)

valid_loader = DataLoader(
    dataset = NsmcDataset(args, 'valid'),
    batch_size = args.batch_size,
    shuffle = True
)

test_loader = DataLoader(
    dataset = NsmcDataset(args, 'test'),
    batch_size = args.batch_size,
    shuffle = True
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model = Model(args)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [10]:
trainer = Trainer(
    args=args,
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader
)

In [11]:
import wandb

wandb.init('Topic Classification')

trainer.fit()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


train epoch: 0: 4688it [58:44,  1.33it/s]
train epoch: 1: 4338it [54:47,  1.32it/s]


KeyboardInterrupt: ignored

In [13]:
model.load('/content/nsmc_ckpt_steps_/2/')

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper

In [16]:
[n for n in model.parameters()][0]

Parameter containing:
tensor([[-0.0117, -0.0293,  0.1731,  ..., -0.0209, -0.0631, -0.0329],
        [-0.0569,  0.0026, -0.0405,  ..., -0.0046, -0.0203,  0.0640],
        [-0.0056, -0.0282,  0.2006,  ..., -0.0207, -0.0455, -0.0347],
        ...,
        [-0.0064,  0.0004,  0.1501,  ..., -0.0410, -0.0615, -0.0575],
        [-0.0406,  0.0249,  0.1766,  ..., -0.0884, -0.0762, -0.0639],
        [-0.0663,  0.0054,  0.1408,  ..., -0.0169, -0.0699, -0.0347]],
       device='cuda:0', requires_grad=True)

In [22]:
model.model.push_to_hub('dhtocks/nsmc_classifier')

CommitInfo(commit_url='https://huggingface.co/dhtocks/nsmc_classifier/commit/3f2f6ca16b582035b5bf7dc278241943e9fa248e', commit_message='Upload BertForSequenceClassification', commit_description='', oid='3f2f6ca16b582035b5bf7dc278241943e9fa248e', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
[n for n in model.model.parameters()][0]

Parameter containing:
tensor([[-0.0117, -0.0293,  0.1731,  ..., -0.0209, -0.0631, -0.0329],
        [-0.0569,  0.0026, -0.0405,  ..., -0.0046, -0.0203,  0.0640],
        [-0.0056, -0.0282,  0.2006,  ..., -0.0207, -0.0455, -0.0347],
        ...,
        [-0.0064,  0.0004,  0.1501,  ..., -0.0410, -0.0615, -0.0575],
        [-0.0406,  0.0249,  0.1766,  ..., -0.0884, -0.0762, -0.0639],
        [-0.0663,  0.0054,  0.1408,  ..., -0.0169, -0.0699, -0.0347]],
       device='cuda:0', requires_grad=True)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized